# Building Multi-Agent Systems with Semantic Kernel

In this notebook, we'll explore how to build multi-agent systems using the Group Chat functionality in Semantic Kernel. We'll create specialized agents that collaborate to solve problems.

## What is an Agent Group Chat?

Agent Group Chat is a collaboration framework that enables multiple specialized agents to:
- Work together on complex tasks
- Take turns in conversation based on defined rules
- Follow specific selection and termination strategies
- Build on each other's outputs to create comprehensive results

Let's start by setting up our environment.

In [ ]:
# Import necessary libraries
import os
import json
import asyncio
from typing import Dict, Any

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.azure_ai_inference import AzureAIInferenceChatCompletion
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.functions import kernel_function, KernelArguments
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.functions import KernelFunctionFromPrompt
from semantic_kernel.contents import ChatHistoryTruncationReducer
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
)

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

## 1. Setting Up Our Environment

First, we'll connect to Azure AI services using the GitHub token for authentication.

In [ ]:
# To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings. 
# Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
base_url="https://models.inference.ai.azure.com"
api_key=os.environ["GITHUB_TOKEN"]
model="gpt-4o-mini"

# Create a chat completion service using Azure OpenAI
chat_completion_service = AzureAIInferenceChatCompletion(
    ai_model_id=model,
    api_key=api_key,
    endpoint=base_url, # Used to point to your service
    service_id="azure_openai", # Optional; for targeting specific services within Semantic Kernel
)

def create_kernel():
    """Create a kernel with Azure OpenAI service"""
    kernel = Kernel()
    
    # Add Azure OpenAI service
    kernel.add_service(chat_completion_service)
    
    return kernel

# Let's create our kernel
kernel = create_kernel()

## 2. Creating Specialized Agents

Now, let's create two specialized agents that will work together in our group chat:

1. **Reviewer** - Analyzes content and provides suggestions for improvement
2. **Writer** - Implements the reviewer's suggestions and revises content

Each agent has specific instructions that define their role and responsibilities.

In [ ]:
REVIEWER_NAME = "Reviewer"
WRITER_NAME = "Writer"

execution_settings = AzureChatPromptExecutionSettings(
    service_id="azure_openai",
    max_tokens=1000,
)

reviewer = ChatCompletionAgent(
    kernel=kernel,
    name=REVIEWER_NAME,
    instructions="""Your responsibility is to review and identify how to improve user provided content.
If the user has provided input or direction for content already provided, specify how to address this input.
Never directly perform the correction or provide an example.
Once the content has been updated in a subsequent response, review it again until it is satisfactory.

RULES:
- Only identify suggestions that are specific and actionable.
- Verify previous suggestions have been addressed.
- Never repeat previous suggestions.",
)
"""
)

copywriter = ChatCompletionAgent(
    kernel=kernel,
    name=WRITER_NAME,
    instructions="""
Your sole responsibility is to rewrite content according to review suggestions.
- Always apply all review directions.
- Always revise the content in its entirety without explanation.
- Never address the user.  
    """,
    arguments=KernelArguments(settings=execution_settings)
)

## 3. Creating the Group Chat

Now we'll create a group chat that includes our specialized agents. In its simplest form, a group chat just needs a list of agents:

In [ ]:
chat = AgentGroupChat(agents=[reviewer, copywriter])

## 4. Adding a Selection Strategy

To make our multi-agent system more effective, we need to define rules for which agent speaks next. Let's create a selection strategy using a kernel function to determine the next participant based on the last message.

In [ ]:
selection_function = KernelFunctionFromPrompt(
    function_name="selection", 
    prompt=f"""
Examine the provided RESPONSE and choose the next participant.
State only the name of the chosen participant without explanation.
Never choose the participant named in the RESPONSE.

Choose only from these participants:
- {REVIEWER_NAME}
- {WRITER_NAME}

Rules:
- If RESPONSE is user input, it is {REVIEWER_NAME}'s turn.
- If RESPONSE is by {REVIEWER_NAME}, it is {WRITER_NAME}'s turn.
- If RESPONSE is by {WRITER_NAME}, it is {REVIEWER_NAME}'s turn.

RESPONSE:
{{{{$lastmessage}}}}
"""
)

## 5. Adding a Termination Strategy

We also need rules for when the conversation should end. Let's create a termination strategy that ends the conversation when the reviewer indicates the content is satisfactory.

In [ ]:
termination_keyword = "yes"

termination_function = KernelFunctionFromPrompt(
    function_name="termination", 
    prompt=f"""
Examine the RESPONSE and determine whether the content has been deemed satisfactory.
If the content is satisfactory, respond with a single word without explanation: {termination_keyword}.
If specific suggestions are being provided, it is not satisfactory.
If no correction is suggested, it is satisfactory.

RESPONSE:
{{{{$lastmessage}}}}
"""
)

## 6. Creating a History Reducer

To manage conversation context efficiently, we'll create a history reducer that limits how much of the conversation history is passed to our selection and termination functions.

In [ ]:
history_reducer = ChatHistoryTruncationReducer(target_count=1)

## 7. Building the Complete Group Chat System

Now we'll put everything together to create our complete group chat system with specialized agents, selection strategy, and termination strategy.

In [ ]:
chat = AgentGroupChat(
    agents=[reviewer, copywriter],
    selection_strategy=KernelFunctionSelectionStrategy(
        initial_agent=reviewer,
        function=selection_function,
        kernel=kernel,
        result_parser=lambda result: str(result.value[0]).strip() if result.value[0] is not None else WRITER_NAME,
        history_variable_name="lastmessage",
        history_reducer=history_reducer,
    ),
    termination_strategy=KernelFunctionTerminationStrategy(
        agents=[reviewer],
        function=termination_function,
        kernel=kernel,
        result_parser=lambda result: termination_keyword in str(result.value[0]).lower(),
        history_variable_name="lastmessage",
        maximum_iterations=10,
        history_reducer=history_reducer,
    ),
)

## 8. Creating a Chat Function

Let's create a function to run our group chat system. This function handles user input, processing, and displaying responses from our agents.

In [ ]:
async def run(user_input: str):
    is_complete = False
    while not is_complete:
        print()

        user_input = input("User > ").strip()
        if not user_input:
            continue

        if user_input.lower() == "exit":
            is_complete = True
            break

        if user_input.lower() == "reset":
            await chat.reset()
            print("[Conversation has been reset]")
            continue

        # Try to grab files from the script's current directory
        if user_input.startswith("@") and len(user_input) > 1:
            file_name = user_input[1:]
            script_dir = os.path.dirname(os.path.abspath(__file__))
            file_path = os.path.join(script_dir, file_name)
            try:
                if not os.path.exists(file_path):
                    print(f"Unable to access file: {file_path}")
                    continue
                with open(file_path, "r", encoding="utf-8") as file:
                    user_input = file.read()
            except Exception:
                print(f"Unable to access file: {file_path}")
                continue

        # Add the current user_input to the chat
        await chat.add_chat_message(message=user_input)

        try:
            async for response in chat.invoke():
                if response is None or not response.name:
                    continue
                print()
                print(f"# {response.name.upper()}:\n{response.content}")
        except Exception as e:
            print(f"Error during chat invocation: {e}")

        # Reset the chat's complete flag for the new conversation round.
        chat.is_complete = False

## 9. Interact with the Group Chat System

Now you can interact with the group chat system! Try entering text content that you'd like to have reviewed and improved.

- Type your content and the Reviewer will suggest improvements
- The Writer will implement those improvements
- The cycle continues until the Reviewer is satisfied
- Type 'exit' to end the conversation
- Type 'reset' to start a new conversation

In [ ]:
is_complete = False
while not is_complete:
    print()

    user_input = input("User > ").strip()
    if not user_input:
        continue

    if user_input.lower() == "exit":
        is_complete = True
        break

    if user_input.lower() == "reset":
        await chat.reset()
        print("[Conversation has been reset]")
        continue

    # Try to grab files from the script's current directory
    if user_input.startswith("@") and len(user_input) > 1:
        file_name = user_input[1:]
        script_dir = os.path.dirname(os.path.abspath(__file__))
        file_path = os.path.join(script_dir, file_name)
        try:
            if not os.path.exists(file_path):
                print(f"Unable to access file: {file_path}")
                continue
            with open(file_path, "r", encoding="utf-8") as file:
                user_input = file.read()
        except Exception:
            print(f"Unable to access file: {file_path}")
            continue

    # Add the current user_input to the chat
    await chat.add_chat_message(message=user_input)

    try:
        async for response in chat.invoke():
            if response is None or not response.name:
                continue
            print()
            print(f"# {response.name.upper()}:\n{response.content}")
    except Exception as e:
        print(f"Error during chat invocation: {e}")

    # Reset the chat's complete flag for the new conversation round.
    chat.is_complete = False


## Example Usage

Here are some example use cases for our Group Chat system:

1. **Content Editing**: Enter an article draft and let the agents improve it through multiple revisions
2. **Technical Documentation**: Enter technical documentation to have it reviewed for clarity and completeness
3. **Email Drafting**: Enter a draft email to have it polished and improved
4. **Marketing Copy**: Enter marketing material to have it refined for impact and clarity

Try this example article about Pluto:

```
Write an article about Pluto.
```

## 10. Challenge: Build Your Own Multi-Agent System

Now it's your turn to build a multi-agent system for a specific task! Here are some ideas:

1. **Research Team**: Create a multi-agent system with a researcher who finds information and an analyst who interprets it
2. **Code Review System**: Create agents for code writing, code reviewing, and testing
3. **Learning Assistant**: Create a teacher agent who provides lessons and a quiz agent who tests understanding
4. **Creative Workshop**: Create a system with a brainstormer, a critic, and a refiner for creative content

For your custom system, you'll need to:
1. Define specialized agents with clear roles
2. Create custom selection and termination strategies
3. Implement a conversation flow

Here's a template to help you get started:

In [ ]:
# Create your specialized agents
agent1 = ChatCompletionAgent(
    kernel=kernel,
    name="Agent1Name",
    instructions="Detailed instructions for agent 1..."
)

agent2 = ChatCompletionAgent(
    kernel=kernel,
    name="Agent2Name",
    instructions="Detailed instructions for agent 2..."
)

# Optional agent3
agent3 = ChatCompletionAgent(
    kernel=kernel,
    name="Agent3Name",
    instructions="Detailed instructions for agent 3..."
)

# Create your selection function
custom_selection = KernelFunctionFromPrompt(
    function_name="custom_selection", 
    prompt="Your custom selection prompt here..."
)

# Create your termination function
custom_termination = KernelFunctionFromPrompt(
    function_name="custom_termination", 
    prompt="Your custom termination prompt here..."
)

# Build your group chat
custom_chat = AgentGroupChat(
    agents=[agent1, agent2, agent3],
    selection_strategy=KernelFunctionSelectionStrategy(
        # Configure your selection strategy
    ),
    termination_strategy=KernelFunctionTerminationStrategy(
        # Configure your termination strategy
    ),
)

# Run your custom chat system
# Similar to the run function above, but adapted for your needs

## 11. Conclusion

In this notebook, we've learned how to:

1. Create specialized agents with defined roles and responsibilities
2. Implement a group chat system that orchestrates agent collaboration
3. Define strategies for agent selection and conversation termination
4. Build a complete multi-agent system that improves content through collaborative review and revision

Multi-agent systems are a powerful way to solve complex problems by breaking them down into specialized roles. By orchestrating collaboration between these agents, we can create systems that produce better results than any single agent could achieve alone.

For more information, visit the [Semantic Kernel documentation](https://learn.microsoft.com/en-us/semantic-kernel/).